# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 2-2. Эвристика Кернигана—Лина

В этой задаче Вам предлагается добавить к локальному поиска в задаче о сбалансированном разбиении графа эвристику Кернигана—Лина, когда мы, «застряв» в локальном минимуме, тем не менее пытаемся сделать несколько шагов из него, даже если они приводят к временному ухудшению. Надежда здесь на то, что после ухудшения может наступить заметное улучшение результата: нам удастся выпрыгнуть из локального оптимума. Мы рассматриваем безвесовый вариант задачи о разбиении с параметром балансировки $\alpha=\frac{1}{2}$:

**Даны:**
* $G=(V,E)$ — граф без весов на рёбрах

**Найти:**
* Разбиение $V=V'\sqcup V''$, такое, что $V'=\lfloor |V|/2 \rfloor$ и число рёбер между $V'$ и $V''$ минимально возможное.

Сделайте следующее:
* Скачайте файл [`partition-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/partition-instances.zip) и разархивируйте из него файлы со входами задачи.
* Для каждого из графов найдите локальным поиском с эвристикой Кернигана—Лина локально минимальное (по количеству рёбер между частями) разбиение вершин графа на две части, мощности которых отличаются не более чем на единицу. 
* Реализуйте функцию `variable_depth_local_search`; она должна принимать на вход граф в формате, предоставляемом функцией `read_instance`, и возвращать найденное разбиение как множество вершин, лежащих в одной любой из двух компонент разбиения. Ваш локальный поиск должен начинаться с того разбиение, которое уже находится в переменной `starting_point`.
* Подберите для каждого из четырёх входных графов глубину поиска так, чтобы он работал не более 60 секунд на Вашем компьютере, и сохраните информацию о подобранных параметрах и любые свои интересные наблюдения в отдельную ячейку настоящего ipynb-файла.

In [14]:
import math

In [15]:
def read_instance(filename):
    with open(filename, 'r') as file:
        n_vertices = int(file.readline().strip().split()[0])
        vertices, edges = set(range(1, n_vertices + 1)), set()
        for u in range(1, n_vertices + 1):
            for v in map(int, file.readline().strip().split()):
                edges.add((u,v))
        return (vertices, edges)

def get_quality(graph, partition_part):
    if not (partition_part <= graph[0]) or abs(len(partition_part) - len(graph[0]) / 2) > 0.6:
        return -1
    other_part = set(graph[0]) - partition_part
    return sum(1 for edge in graph[1] if set(edge) <= partition_part or set(edge) <= other_part )


In [16]:
def cut_cost(L: set, R: set, edges:set) -> int:
    ans = 0
    for edge in edges:
        v1, v2 = edge
        if (v1 in L and v2 in R) or (v1 in R and v2 in L):
            ans += 1
    return ans

# returns best local(1-dist area) partition
def best_local(graph, L, R):
    exchange_cost = [0 for i in range(len(graph[0]))]
    for u, v in graph[1]:
        if (u in L and v in R) or (u in R and v in L):
            exchange_cost[u - 1] -= 1
            exchange_cost[v - 1] -= 1
        else:
            exchange_cost[u - 1] += 1
            exchange_cost[v - 1] += 1
    
    best_exchange_cost = math.inf
    best_v1 = -1
    best_v2 = -1
    
    for v1 in L:
        for v2 in R:
            is_edge = (v1, v2) in graph[1] or (v2, v1) in graph[1]
            
            cur_exchange_cost = exchange_cost[v1 - 1] + exchange_cost[v2 - 1] + 2 * is_edge
            if cur_exchange_cost < best_exchange_cost:
                best_exchange_cost = cur_exchange_cost
                best_v1 = v1
                best_v2 = v2
    
    best_L = L.copy()
    best_R = R.copy()
    
    best_L.remove(best_v1)
    best_L.add(best_v2)
    best_R.remove(best_v2)
    best_R.add(best_v1)
    
    return best_L, best_R
            

def variable_depth_local_search(graph: tuple, max_depth: int) -> set:
    
    vertices, edges = graph
    n = len(vertices)
    
    starting_point = set(range(1, len(graph[0]) // 2 + 1))
    L = starting_point
    R = vertices - starting_point
    
    depth = 0
    best_cut_cost = cut_cost(L, R, graph[1])
    best_L = L
    while depth < max_depth:
        best_local_L, best_local_R = best_local(graph, L, R)
        new_cut_cost = cut_cost(best_local_L, best_local_R, edges)
        if new_cut_cost >= best_cut_cost:
            depth += 1
            print("Local maximum:", get_quality(graph, best_L))
        else:
            best_cut_cost = new_cut_cost
            depth = 0
            best_L = best_local_L
        L = best_local_L
        R = best_local_R
    
    return best_L

In [25]:
import time

def run_all():
    filenames = ['add20.graph', 'cti.graph', 't60k.graph']#, 'm14b.graph']
    for max_depth in range(5):
        print('\n=============Depth {}============='.format(max_depth))
        for filename in filenames:
            instance = read_instance(filename)
            print('Solving instance {}'.format(filename))
            time_start = time.monotonic()

            quality = get_quality(instance, variable_depth_local_search(instance, max_depth))

            time_elapsed = time.monotonic() - time_start
            print('Done in {:.2} seconds with quality {}'.format(time_elapsed, quality))

In [26]:
run_all()


=============Depth 0=============
Solving instance add20.graph
Done in 0.011 seconds with quality 11070
Solving instance cti.graph
Done in 0.081 seconds with quality 94052
Solving instance t60k.graph
Done in 0.19 seconds with quality 178236

=============Depth 1=============
Solving instance add20.graph
Local maximum: 12462
Done in 1.3e+02 seconds with quality 12462
Solving instance cti.graph
Local maximum: 94080
Done in 2e+02 seconds with quality 94080
Solving instance t60k.graph
Local maximum: 178248
Done in 1.7e+03 seconds with quality 178248

=============Depth 2=============
Solving instance add20.graph
Local maximum: 12462
Local maximum: 12528
Local maximum: 12554
Local maximum: 12580
Local maximum: 12584
Local maximum: 12588
Local maximum: 12622
Local maximum: 12626
Local maximum: 12630
Local maximum: 12672
Local maximum: 12710
Local maximum: 12714
Local maximum: 12718
Local maximum: 12722
Local maximum: 12726
Local maximum: 12730
Local maximum: 12734
Local maximum: 12762
Local

## Выводы


Как мы видим, увеличение глубины выше 2 уже не приводит к сильному улучшению
(на тесте m14b я запустил на всю ночь, но оно не успело отработать, так что я решил, что это плохая идея), так что глубину 2 предлагаю считать оптимальной.